# --- Day 9: Mirage Maintenance ---
Analyze your OASIS report and extrapolate the next value for each history. What is the sum of these extrapolated values?




In [2]:
import numpy as np

In [72]:
infile = "input.0.txt"

In [63]:
m = np.loadtxt(infile, dtype=int)
m.shape
n

array([[ 0,  3,  6,  9, 12, 15],
       [ 1,  3,  6, 10, 15, 21],
       [10, 13, 16, 21, 30, 45]])

In [85]:
def predict(a, s=0):

    #print("f",a, s)
    if np.all(np.isclose(a, 0)):
        return s
        
    dif = a[1:] - a[:-1]
    s = predict(dif, s)

    s += a[-1]
    return s

In [67]:
np.apply_along_axis(predict, 1, m).sum()

114

In [68]:
infile = "input.txt"
m = np.loadtxt(infile, dtype=int)

In [69]:
np.apply_along_axis(predict, 1, m).sum()

1921197370

## --- Part Two ---
Analyze your OASIS report again, this time extrapolating the previous value for each history. What is the sum of these extrapolated values?

In [73]:
infile = "input.0.txt"
m = np.loadtxt(infile, dtype=int)

In [74]:
m

array([[ 0,  3,  6,  9, 12, 15],
       [ 1,  3,  6, 10, 15, 21],
       [10, 13, 16, 21, 30, 45]])

In [94]:
def predict_left(a, s=0):

    #print("f",a, s)
    if np.all(np.isclose(a, 0)):
        return s
        
    dif = a[1:] - a[:-1]
    s = predict_left(dif, s)

    s = a[0] - s
    return s

predict_left(m[2])

5

In [96]:
np.apply_along_axis(predict_left, 1, m).sum()

2

In [97]:
infile = "input.txt"
m = np.loadtxt(infile, dtype=int)
np.apply_along_axis(predict_left, 1, m).sum()

1124